In [42]:
"""
Title: Eating disorders Montecatini 
Author: Corrado Caudek
Description: This notebook computes the effect size (Cohen's d) for 
    all within-group and between groups comparisons. The DV are the 
    hDDMrl parameters estimated for each participant.
Data Source: '/Users/corrado/_repositories/eating_disorders_23/data/processed/prl/subj_hddm_params/subj_code_hddm_params.csv'
Version History:
- v1.0 (July 6, 2023): Initial version
- v1.1 (July 7, 2023): Updated computation Cohen's d
Contact: corrado.caudek@unifi.it
"""

# Virtual environment: py37_env

import datetime

now = datetime.datetime.now()
print("Current date and time : ")
print(now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time : 
2023-07-07 06:41:49


In [43]:
import os, time, csv, sys

import numpy as np
import pandas as pd
from scipy import stats

# Data management
pd.options.display.max_colwidth = 100


In [44]:
working_directory = "/Users/corrado/_repositories/eating_disorders_23/"

file_path = os.path.join(
    working_directory,
    "data",
    "processed",
    "prl",
    "subj_hddm_params",
    "subj_code_hddm_params.csv",
)

df = pd.read_csv(file_path)
df.head()

,params,val,subj_idx,par,diag_cat,stim,subj_code
0,a_subj(AN.food).1,1.270092,1,a,AN,food,ca_po_2002_05_25_700_f
1,a_subj(AN.neutral).1,1.131988,1,a,AN,neutral,ca_po_2002_05_25_700_f
2,v_subj(AN.food).1,0.799608,1,v,AN,food,ca_po_2002_05_25_700_f
3,v_subj(AN.neutral).1,0.900103,1,v,AN,neutral,ca_po_2002_05_25_700_f
4,t_subj(AN.food).1,0.088883,1,t,AN,food,ca_po_2002_05_25_700_f


In [45]:
file_path

'/Users/corrado/_repositories/eating_disorders_23/data/processed/prl/subj_hddm_params/subj_code_hddm_params.csv'

## Cohen's d

In [46]:
mean_df = df.groupby(["par", "diag_cat", "stim"])["val"].mean().reset_index()
print(mean_df.round(3))

          par diag_cat     stim    val
0           a       AN     food  1.447
1           a       AN  neutral  1.271
2           a       HC     food  1.311
3           a       HC  neutral  1.214
4           a       RI     food  1.334
5           a       RI  neutral  1.239
6       alpha       AN     food  0.761
7       alpha       AN  neutral  1.903
8       alpha       HC     food  3.581
9       alpha       HC  neutral  2.886
10      alpha       RI     food  3.045
11      alpha       RI  neutral  3.008
12  pos_alpha       AN     food -1.838
13  pos_alpha       AN  neutral  1.041
14  pos_alpha       HC     food  1.352
15  pos_alpha       HC  neutral  1.217
16  pos_alpha       RI     food  0.762
17  pos_alpha       RI  neutral  1.622
18          t       AN     food  0.180
19          t       AN  neutral  0.205
20          t       HC     food  0.162
21          t       HC  neutral  0.177
22          t       RI     food  0.188
23          t       RI  neutral  0.176
24          v       AN   

In [47]:
mean_df = df.groupby(["par", "diag_cat", "stim"])["val"].std().reset_index()
print(mean_df.round(3))

          par diag_cat     stim    val
0           a       AN     food  0.233
1           a       AN  neutral  0.205
2           a       HC     food  0.208
3           a       HC  neutral  0.201
4           a       RI     food  0.265
5           a       RI  neutral  0.198
6       alpha       AN     food  2.852
7       alpha       AN  neutral  2.403
8       alpha       HC     food  2.128
9       alpha       HC  neutral  2.525
10      alpha       RI     food  2.520
11      alpha       RI  neutral  2.277
12  pos_alpha       AN     food  2.423
13  pos_alpha       AN  neutral  2.352
14  pos_alpha       HC     food  1.881
15  pos_alpha       HC  neutral  1.929
16  pos_alpha       RI     food  1.848
17  pos_alpha       RI  neutral  2.081
18          t       AN     food  0.081
19          t       AN  neutral  0.115
20          t       HC     food  0.052
21          t       HC  neutral  0.058
22          t       RI     food  0.088
23          t       RI  neutral  0.046
24          v       AN   

In [48]:
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

### Stimulus effects

In [49]:
def compute_cohens_d(df, stim_col, diag_cat_col, par_col, val_col):
    results = pd.DataFrame(columns=["diag_cat", "par", "cohens_d"])

    # Iterate over unique levels of diag_cat
    diag_cat_levels = df[diag_cat_col].unique()
    for diag_cat_level in diag_cat_levels:
        # Filter the DataFrame for the current diag_cat level
        diag_cat_df = df[df[diag_cat_col] == diag_cat_level]

        # Iterate over unique levels of par
        par_levels = diag_cat_df[par_col].unique()
        for par_level in par_levels:
            # Filter the DataFrame for the current par level
            par_df = diag_cat_df[diag_cat_df[par_col] == par_level]

            # Filter the DataFrame for the two stim levels
            stim1_df = par_df[par_df[stim_col] == "food"]
            stim2_df = par_df[par_df[stim_col] == "neutral"]

            # Calculate Cohen's d using the cohend function
            cohens_d = cohend(stim1_df[val_col].values, stim2_df[val_col].values)

            # Append the results to the DataFrame
            results = results.append(
                {"diag_cat": diag_cat_level, "par": par_level, "cohens_d": cohens_d},
                ignore_index=True,
            )

    return results


result_df = compute_cohens_d(df, "stim", "diag_cat", "par", "val")
print(result_df.round(3))

   diag_cat        par  cohens_d
0        AN          a     0.802
1        AN          v     0.190
2        AN          t    -0.253
3        AN      alpha    -0.432
4        AN  pos_alpha    -1.206
5        HC          a     0.474
6        HC          v    -0.358
7        HC          t    -0.280
8        HC      alpha     0.298
9        HC  pos_alpha     0.071
10       RI          a     0.403
11       RI          v    -0.381
12       RI          t     0.166
13       RI      alpha     0.015
14       RI  pos_alpha    -0.438


### Comparisons between the AN and HC groups

In [50]:
def compute_cohens_d(df, stim_col, par_col, val_col):
    results = pd.DataFrame(columns=["stim", "par", "cohens_d"])

    # Filter the DataFrame for levels 'HC' and 'AN' of diag_cat
    filtered_df = df[df["diag_cat"].isin(["HC", "AN"])]

    # Iterate over unique levels of stim
    stim_levels = filtered_df[stim_col].unique()
    for stim_level in stim_levels:
        # Filter the DataFrame for the current stim level
        stim_df = filtered_df[filtered_df[stim_col] == stim_level]

        # Iterate over unique levels of par
        par_levels = stim_df[par_col].unique()
        for par_level in par_levels:
            # Filter the DataFrame for the current par level
            par_df = stim_df[stim_df[par_col] == par_level]

            # Get the data for 'HC' and 'AN' levels
            hc_data = par_df[par_df["diag_cat"] == "HC"][val_col].values
            an_data = par_df[par_df["diag_cat"] == "AN"][val_col].values

            # Calculate Cohen's d using the cohend function
            cohens_d = cohend(hc_data, an_data)

            # Append the results to the DataFrame
            results = results.append(
                {"stim": stim_level, "par": par_level, "cohens_d": cohens_d},
                ignore_index=True,
            )

    return results


result_df = compute_cohens_d(df, "stim", "par", "val")
print(result_df.round(3))

      stim        par  cohens_d
0     food          a    -0.622
1     food          v     0.007
2     food          t    -0.278
3     food      alpha     1.144
4     food  pos_alpha     1.498
5  neutral          a    -0.279
6  neutral          v     0.520
7  neutral          t    -0.325
8  neutral      alpha     0.397
9  neutral  pos_alpha     0.083


### Comparisons between the AN and RI groups

In [51]:
def compute_cohens_d(df, stim_col, par_col, val_col):
    results = pd.DataFrame(columns=["stim", "par", "cohens_d"])

    # Filter the DataFrame for levels 'HC' and 'AN' of diag_cat
    filtered_df = df[df["diag_cat"].isin(["RI", "AN"])]

    # Iterate over unique levels of stim
    stim_levels = filtered_df[stim_col].unique()
    for stim_level in stim_levels:
        # Filter the DataFrame for the current stim level
        stim_df = filtered_df[filtered_df[stim_col] == stim_level]

        # Iterate over unique levels of par
        par_levels = stim_df[par_col].unique()
        for par_level in par_levels:
            # Filter the DataFrame for the current par level
            par_df = stim_df[stim_df[par_col] == par_level]

            # Get the data for 'HC' and 'AN' levels
            hc_data = par_df[par_df["diag_cat"] == "RI"][val_col].values
            an_data = par_df[par_df["diag_cat"] == "AN"][val_col].values

            # Calculate Cohen's d using the cohend function
            cohens_d = cohend(hc_data, an_data)

            # Append the results to the DataFrame
            results = results.append(
                {"stim": stim_level, "par": par_level, "cohens_d": cohens_d},
                ignore_index=True,
            )

    return results


result_df = compute_cohens_d(df, "stim", "par", "val")
print(result_df.round(3))

      stim        par  cohens_d
0     food          a    -0.454
1     food          v     0.043
2     food          t     0.091
3     food      alpha     0.849
4     food  pos_alpha     1.209
5  neutral          a    -0.158
6  neutral          v     0.529
7  neutral          t    -0.332
8  neutral      alpha     0.472
9  neutral  pos_alpha     0.261


### Comparisons between the HC and RI groups

In [52]:
def compute_cohens_d(df, stim_col, par_col, val_col):
    results = pd.DataFrame(columns=["stim", "par", "cohens_d"])

    # Filter the DataFrame for levels 'HC' and 'AN' of diag_cat
    filtered_df = df[df["diag_cat"].isin(["RI", "HC"])]

    # Iterate over unique levels of stim
    stim_levels = filtered_df[stim_col].unique()
    for stim_level in stim_levels:
        # Filter the DataFrame for the current stim level
        stim_df = filtered_df[filtered_df[stim_col] == stim_level]

        # Iterate over unique levels of par
        par_levels = stim_df[par_col].unique()
        for par_level in par_levels:
            # Filter the DataFrame for the current par level
            par_df = stim_df[stim_df[par_col] == par_level]

            # Get the data for 'HC' and 'AN' levels
            hc_data = par_df[par_df["diag_cat"] == "RI"][val_col].values
            an_data = par_df[par_df["diag_cat"] == "HC"][val_col].values

            # Calculate Cohen's d using the cohend function
            cohens_d = cohend(hc_data, an_data)

            # Append the results to the DataFrame
            results = results.append(
                {"stim": stim_level, "par": par_level, "cohens_d": cohens_d},
                ignore_index=True,
            )

    return results


result_df = compute_cohens_d(df, "stim", "par", "val")
print(result_df.round(3))

      stim        par  cohens_d
0     food          a     0.097
1     food          v     0.059
2     food          t     0.371
3     food      alpha    -0.232
4     food  pos_alpha    -0.316
5  neutral          a     0.124
6  neutral          v     0.041
7  neutral          t    -0.021
8  neutral      alpha     0.050
9  neutral  pos_alpha     0.203
